1-VERİ HAZIRLAMA AŞAMASI


Gerekli kütüphaneleri çağıralım 

In [ ]:
#Genel komutlar

import numpy as np

import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
#RMSE ile tahmin hatalarımı belirlemek için sqrt çağırdım.(evaluate forecast)
from math import sqrt
from sklearn.metrics import mean_squared_error
#Verisetini ayıklamak için çağırdım
from numpy import split
from numpy import array

#LSTM MMODELLETİM İÇİN GEREKLİ KERAS KÜTÜPHANELERİM
from keras.optimizers import Adam
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

In [ ]:
path = "../input/solar-radiation-dataset/2017_2019.csv"
df = pd.read_csv(path)

Verisetini oluşturuken oluşan boş sütunu sildim

In [ ]:
df=df.drop(['Unnamed: 18',"DHI","DNI","Clearsky DHI" ,"Clearsky DNI","Clearsky GHI"], axis = 1) 


-Zaman sütunlarını birleştirip datetime'a çevirdim, sonra diğer sütunları sildim.

-Datetime sütununu index yaptım .

In [ ]:
cols = ["Year","Month","Day","Hour","Minute"]
df["date_time"] = df[cols].apply(lambda row: "-".join(row.values.astype(str)), axis=1)

In [ ]:
df['date_time'] = pd.to_datetime(df['date_time'], format='%Y-%m-%d-%H-%M')

In [ ]:
df=df.drop(["Year","Month","Day","Hour","Minute"], axis = 1)

In [ ]:
df=df.set_index('date_time') #Columnu index yapmak için

Tahmin etmek istediğim değeri son sütuna alıyorum.

In [ ]:
df.head()

In [ ]:
df1 = df.pop('GHI') # GHI sütununu sil ve df1 içine kaydet.
df['GHI']=df1 # GHI  serisini yeni bir sütun olarak sona ekle.


Verisetini LSTM modelinde öğrenilecek şekile getirmek için 3 boyutlu hale getirmem lazım. Tek bir inputta

In [ ]:
# split into days
from numpy import split
from numpy import array

 
# split a univariate dataset into train/test sets
def split_dataset(df):
  train=df[100608:104448]

  test=df[104448:]
  train = array(split(train, len(train)/4))#günlük bölme yaptığım için 96 aldım.
  test = array(split(test, len(test)/4))
  return train, test

Keras kütüphanesi ile hazırlanacak neural networklerde inputları numpy array olarak ayırmak zorundayız.

Sinir ağları modelleri numpy arrayi olarak işlenir.

In [ ]:
train, test = split_dataset(df.values)#verisetini yukarıda yazılan fonksiyon içinde ayırdım.

Her bir günde 15 dk'lık 96 ölçüm var.Modeli denerken işlemi hızlandırmak için şimdilik 40 günlük bir öğrenme veri seti oluşturdum.Her bir satırda 13 özellik(feature) var.Bu yüzden modele okutacağımız öğrenme şeklini 40,96,13  olarak oluşturdum.
Test setini de küçük tutmak adına 1 hafta sonrası olarak aldım.7,96,13

In [ ]:
print('train shape == {}.'.format(train.shape))
print('test shape == {}.'.format(test.shape))


Her bir çıktıya rmse uygulama işlemi.

In [ ]:
def evaluate_forecasts(actual, predicted):
	scores = list()
	# Her bir gerçek (test) değerini döngüye sokup rmse değerini ölçmek için yazılmıştır.
	for i in range(actual.shape[1]):
		# calculate mse
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# calculate rmse
		rmse = sqrt(mse)
		# store
		scores.append(rmse)#hepsini score dosyasına kaydet
	# calculate overall RMSE
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores

In [ ]:
# Skoru (Rmse değerlerini) görselleştirme
def summarize_scores(name, score, scores):
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))

Verisetinde bir trend yakalaması için  uygun timestep(zamanaralığı) değeri ayarlamaya çalıştım.

!!!! BURASINI OPTİMİZE ETMELİYİM!!!!

Ne kadar bir zamanı tahmin etmeliyim (96 data=1 gün).

Hangi zaman aralığını input hangisini output almalıyım?
Solar radyasyon verisi sabah 8 akşam 5 aralığında ölçülüyor genelde. Bu saatler dışında radyasyon sıfıra çok yakın . bu yüzden 1 günü traine (X) ve bu günün sabahını outputa koyarsam(y) saçma bir  sonuca ulaşabilirim. 

In [ ]:

# input ve output değerlerini ayarlama
def to_supervised(train, n_input, n_out=4):
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			X.append(data[in_start:in_end, :])
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

Modeldi hızlandırmak veya iyileştirmek için uygulanabilecek şeyler. 

-nöron sayısını değiştirmek

-dropout layerı eklemek

-aktivasyon fonksiyonlarını değiştirmek

-optimizerı değiştirmek.(adam ideal olabilir )

#Eklemem gerek!:
-Daha sağlıklı bir analiz için model süresi . 
Loss grafiği(gradient descent minimuma yaklaşıyor mu ? )
-tensorflowda bir model kurmak  durumu hızlandırabilir bunu da denemem lazım .
#Warning 
Sistemin cuDNN e göre çalışmaması modeli yavaşlatıyor sanırım .

-( Layer lstm_13 will not use cuDNN kernel since it doesn't meet the cuDNN kernel criteria. It will use generic GPU kernel as fallback when running on GPU)

In [ ]:
# train the model
def build_model(train, n_input):


	# prepare data
	train_x, train_y = to_supervised(train, n_input)
	# define parameters
	verbose, epochs, batch_size = 1, 50, 16
  #timesteps bizim belirlediğimiz 96 değeri(1 gün)
  #feature ,özellik sayısı = 13
  #çıktı,(n_output) değeri y nin 2. değeri 
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# reshape output into [samples, timesteps, features]
	train_y = train_y.reshape((train_y.shape[0], train_y.shape[1], 1))
	# define model
	model = Sequential()
	model.add(LSTM(128, activation='relu', input_shape=(n_timesteps, n_features))) #giriş değerini 200 nöronla deneme
	model.add(RepeatVector(n_outputs)) # iki boyutlu girdiyi 3 boyuta çevirme komutu.input_shape=(n_timesteps,n_outputs, n_features)olur
	model.add(LSTM(128,  activation='relu',return_sequences=True))
	model.add(TimeDistributed(Dense(64, activation='relu')))#time distribute girdi ve çıktısı 3 D olmalıdır.
	model.add(TimeDistributed(Dense(1)))

	model.compile(optimizer = Adam(learning_rate=0.01), loss='mean_squared_error')
	# fit network
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

In [ ]:
def forecast(model, history, n_input):
	# flatten data
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data
	input_x = data[-n_input:, :]
	# reshape into [1, n_input, n]
	input_x = input_x.reshape((1, input_x.shape[0], input_x.shape[1]))
	# forecast the next week
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast
	yhat = yhat[0]
	return yhat
 

In [ ]:
# modeli train üstünde çalıştırıp,test ile karşılaştırmak
def evaluate_model(train, test, n_input):
	
	model = build_model(train, n_input)
	model.history.history.keys()
	history = [x for x in train]
	myloss = model.history.history["loss"]

	predictions = list()
	for i in range(len(test)):
		# Traini tahmin etme ve y headi oluşturma
		yhat_sequence = forecast(model, history, n_input)
		# tahminleri kaydetme
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week
    #test verilerinin indexe göre dizilişi ve tahmin
		history.append(test[i, :])
	# evaluate predictions hours for each day
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores

In [ ]:
n_input = 4
score, scores = evaluate_model(train, test, n_input)

In [ ]:
summarize_scores('lstm:rmse ortalama', score, scores)